In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('/content/drive/MyDrive/jigsaw-text-data/train_clean.csv')
test = pd.read_csv('/content/drive/MyDrive/jigsaw-text-data/test_clean.csv')
train_text = np.loadtxt('/content/drive/MyDrive/jigsaw-text-data/train_text.csv', delimiter=',')
test_text = np.loadtxt('/content/drive/MyDrive/jigsaw-text-data/test_text.csv', delimiter=',')
y = train.iloc[:,2:]

In [4]:
import string
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [5]:
maxlen = 200
max_features = 5000

inp = Input(shape=(maxlen, ))
embed_size = 128
x = Embedding(max_features, embed_size)(inp)
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)

In [6]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [7]:
batch_size = 32
epochs = 3
model.fit(train_text,y, batch_size=batch_size, epochs=epochs, validation_split=0.15)

Epoch 1/3
4239/4239 [==============================] - 195s 44ms/step - loss: 0.0734 - accuracy: 0.9374 - val_loss: 0.0532 - val_accuracy: 0.9943
Epoch 2/3
4239/4239 [==============================] - 188s 44ms/step - loss: 0.0506 - accuracy: 0.9898 - val_loss: 0.0507 - val_accuracy: 0.9942
Epoch 3/3
4239/4239 [==============================] - 188s 44ms/step - loss: 0.0461 - accuracy: 0.9914 - val_loss: 0.0510 - val_accuracy: 0.9940


In [8]:
submission = pd.read_csv('/content/drive/MyDrive/jigsaw-text-data/sample_submission.csv')

In [9]:
from sklearn.metrics import roc_curve, roc_auc_score
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
preds = model.predict(test_text, batch_size=1024)
submission[list_classes] = preds

In [11]:
submission.to_csv('/content/drive/MyDrive/jigsaw-text-data/sample_submission_1.csv',index=False)